<a href="https://colab.research.google.com/github/elyte5star/Uti-AI/blob/master/DEEP_LEARNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Twitter Classification Problem | Predicting the sentiment of tweets

<h1>Introduction</h1>
![title](https://www.pngkit.com/png/detail/25-251946_twitter-link-to-logo-twitter.png)

In [6]:
# Code is compatible with Python 3.X
import warnings
from string import digits
import string
from tensorflow.keras.layers import Dense
import os
import re
import pandas as pd
import sys
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from tensorflow import keras
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10,10)
mpl.rcParams['axes.grid'] = False
import numpy as np
np.random.seed(42)
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.linear_model import Perceptron
import time
import functools
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
warnings.filterwarnings(action="ignore", message="^internal gelsd")
print("Running Panda Version:"+pd.__version__)
print("Running TensorFlow Version:"+ tf.__version__)
print("Running Keras API Version:"+ keras.__version__)
print("Running Python {0}.{1}".format(sys.version_info[:2][0],sys.version_info[:2][1]))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Running Panda Version:0.25.3
Running TensorFlow Version:1.15.0
Running Keras API Version:2.2.4-tf
Running Python 3.6


In [7]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"elyte5star","key":"aac121888b41b7b4a4f6e07bb0615b4e"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [9]:
!kaggle competitions download -c cs98x-Twitter-Sentiment --force

 72% 41.0M/57.3M [00:01<00:01, 15.6MB/s]
100% 57.3M/57.3M [00:01<00:00, 35.0MB/s]
 75% 17.0M/22.8M [00:00<00:00, 18.1MB/s]
100% 22.8M/22.8M [00:00<00:00, 24.7MB/s]


In [10]:
!unzip "/content/training.csv.zip"
!unzip "/content/test.csv.zip"

Archive:  /content/training.csv.zip
  inflating: training.csv            
Archive:  /content/test.csv.zip
  inflating: test.csv                


In [0]:
#Load datasets
Train = pd.read_csv('/content/training.csv',na_values=['NA'],low_memory=False)# making data frame from csv file(Train dataset) 
Test = pd.read_csv('/content/test.csv',low_memory=False)# making data frame from csv file(Test dataset) 

In [0]:
print(f'Train dataset has {Train.shape[0]} rows/observation and {Train.shape[1]} columns/features.')

In [0]:

Train = Train.set_index('id')#set the Id column as the index

In [13]:
print(f'Test dataset has {Test.shape[0]} rows/observation and {Test.shape[1]} columns7features.')

Train dataset has 1000000 rows/observation and 4 columns/features.
Test dataset has 399890 rows/observation and 4 columns7features.


In [14]:
Train.head()#shows the first five observations of the train dataset.

,target,date,user,text
id,,,,
2224496154,0,Thu Jun 18 09:34:25 PDT 2009,Deborah_Fay,Okay for some reason I still can't get the ba...
1687934301,4,Sun May 03 09:56:56 PDT 2009,haills,http://twitpic.com/4hjnj - Don't worry be happ...
1972073814,0,Sat May 30 08:30:21 PDT 2009,lechellealice,this movie is so sad
1557632618,4,Sun Apr 19 05:13:47 PDT 2009,janeslee,Good afternoon world. Been too busy to tweet. ...
1969750122,4,Sat May 30 01:05:20 PDT 2009,bvdaily,@QueensleyFelix LOL!We are just too classy I ...


In [15]:
Test.head()#shows the first five observations of the Test dataset.

,id,date,user,text
0,1974447176,Sat May 30 13:09:18 PDT 2009,ugro,That's pretty much how I think it works with i...
1,1752584531,Sat May 09 22:20:24 PDT 2009,infinities,Holy awesome Leonard Nimoy! Also thought I sp...
2,2265878673,Sun Jun 21 07:29:15 PDT 2009,revybbcholic,@theodorainish yes they are I've tried to not...
3,1995188099,Mon Jun 01 13:05:22 PDT 2009,teddyrised,Millvina Dean the last known survivor of the ...
4,2071942859,Sun Jun 07 20:08:43 PDT 2009,kaylabeth_,I REALLY DO NOT LIKE THIS ROGER. Adam Pascal p...


In [16]:
print(Train.columns.values)#names of columns in the train dataset

['target' 'date' 'user' 'text']


In [17]:
print(Test.columns.values)#Same as the train dataset except the target column missing

['id' 'date' 'user' 'text']


In [0]:
Test.columns = ['id','date', 'user', 'text'] #Rename the columns
Test = Test.set_index('id')#set the Id column as the index

In [19]:
Test.head()#shows the first five observations of the Test dataset.

,date,user,text
id,,,
1974447176,Sat May 30 13:09:18 PDT 2009,ugro,That's pretty much how I think it works with i...
1752584531,Sat May 09 22:20:24 PDT 2009,infinities,Holy awesome Leonard Nimoy! Also thought I sp...
2265878673,Sun Jun 21 07:29:15 PDT 2009,revybbcholic,@theodorainish yes they are I've tried to not...
1995188099,Mon Jun 01 13:05:22 PDT 2009,teddyrised,Millvina Dean the last known survivor of the ...
2071942859,Sun Jun 07 20:08:43 PDT 2009,kaylabeth_,I REALLY DO NOT LIKE THIS ROGER. Adam Pascal p...


In [20]:
Train.info()#quick description of the data

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 2224496154 to 1795408948
Data columns (total 4 columns):
target    1000000 non-null int64
date      1000000 non-null object
user      1000000 non-null object
text      1000000 non-null object
dtypes: int64(1), object(3)
memory usage: 38.1+ MB


## Removing duplicates

In [0]:
boolean_duplicates_train = Train.duplicated().value_counts()#check for duplicates in the train dataset
boolean_duplicates_test = Test.duplicated().value_counts()#check for duplicates in the test dataset

In [22]:
boolean_duplicates_train#Returns True for duplicate in train dataset

False    999908
True         92
dtype: int64

In [23]:
boolean_duplicates_test##Returns True for duplicate in test dataset

False    399881
True          9
dtype: int64

In [0]:
DuplicateRows = Train[Train.duplicated(keep='first')]#Saves duplicate rows in the variable duplicateRows

In [25]:
DuplicateRows# Returns duplicate row in the train dataset

,target,date,user,text
id,,,,
1976984787,4,Sat May 30 19:47:03 PDT 2009,maricrisg,@Momsmostwanted are we allowed to win 2x or 3x...
1983476191,0,Sun May 31 13:13:45 PDT 2009,disneygirl92,Hey @souljaboytellem Soulja Come to SCOTLAND ...
1975320086,0,Sat May 30 14:54:41 PDT 2009,violetbakes,@pawpads I know I just caught it scrolling ac...
1881781718,4,Fri May 22 05:32:40 PDT 2009,virtuosoencore,@a_rybak Welcome to Munich.
1985821752,4,Sun May 31 17:50:18 PDT 2009,lina003,is drinking Barq's and getting ready to eat so...
...,...,...,...,...
2219490899,0,Thu Jun 18 00:39:36 PDT 2009,HeatherHero,my phone's gone for the whole summer break
1972952949,0,Sat May 30 10:11:48 PDT 2009,king_of_rock,Ferry from cowes 2 so'ton! Hols ova...
1969641371,4,Sat May 30 00:42:31 PDT 2009,Mig177,Hmm Which twitter application is faster? Tweet...


In [26]:
Train.drop_duplicates()#drop duplicate rows

,target,date,user,text
id,,,,
2224496154,0,Thu Jun 18 09:34:25 PDT 2009,Deborah_Fay,Okay for some reason I still can't get the ba...
1687934301,4,Sun May 03 09:56:56 PDT 2009,haills,http://twitpic.com/4hjnj - Don't worry be happ...
1972073814,0,Sat May 30 08:30:21 PDT 2009,lechellealice,this movie is so sad
1557632618,4,Sun Apr 19 05:13:47 PDT 2009,janeslee,Good afternoon world. Been too busy to tweet. ...
1969750122,4,Sat May 30 01:05:20 PDT 2009,bvdaily,@QueensleyFelix LOL!We are just too classy I ...
...,...,...,...,...
1979073420,4,Sun May 31 01:31:46 PDT 2009,xxxSupermodel,@thomaslawler he's adorable. Hi Jimmy
2207066448,0,Wed Jun 17 07:26:23 PDT 2009,emmavescence,@DJ_Alexander It's really bad today I've take...
2194236338,0,Tue Jun 16 09:31:27 PDT 2009,1rdreamer,Can't vote at the moment coming up &quot;inte...


In [0]:
DuplicateRows_test = Test[Test.duplicated(keep='first')]#Saves duplicate rows in the variable duplicateRows_test

In [28]:
DuplicateRows_test

,date,user,text
id,,,
2202016940,Tue Jun 16 21:06:49 PDT 2009,Viper_Saint,Can't belive the alarm has gone off already. I...
1976960301,Sat May 30 19:42:16 PDT 2009,pynacolada,@NIKKI68 hi! I was working in marketing for so...
2226844279,Thu Jun 18 12:27:29 PDT 2009,lex2324,the last month has been a nightmare i miss my ...
1770606391,Mon May 11 21:46:35 PDT 2009,lilymcnee,twitter isnt workin properly not fun when ur...
1962008906,Fri May 29 10:22:54 PDT 2009,kellebelle1981,i hope this works. it gave me lots of joy. h...
1964417712,Fri May 29 14:12:12 PDT 2009,Xo_Abby_oX,good day 2day hot weather and great friends wh...
1990215016,Mon Jun 01 04:13:54 PDT 2009,Buttonmashers,Too damn hot in Gothenburg today need a cold ...
1964815104,Fri May 29 14:51:09 PDT 2009,otaking,@IsobelWren Yes?
2326023952,Thu Jun 25 06:26:14 PDT 2009,AlexandraG730,@Ashley_Cashen Yes! I have to see you before y...


In [29]:
Test.drop_duplicates()#drop duplicate rows in test

,date,user,text
id,,,
1974447176,Sat May 30 13:09:18 PDT 2009,ugro,That's pretty much how I think it works with i...
1752584531,Sat May 09 22:20:24 PDT 2009,infinities,Holy awesome Leonard Nimoy! Also thought I sp...
2265878673,Sun Jun 21 07:29:15 PDT 2009,revybbcholic,@theodorainish yes they are I've tried to not...
1995188099,Mon Jun 01 13:05:22 PDT 2009,teddyrised,Millvina Dean the last known survivor of the ...
2071942859,Sun Jun 07 20:08:43 PDT 2009,kaylabeth_,I REALLY DO NOT LIKE THIS ROGER. Adam Pascal p...
...,...,...,...
2256218745,Sat Jun 20 12:26:31 PDT 2009,nightwheel,I see you mike on the live show. Wishing I cou...
2265880544,Sun Jun 21 07:29:28 PDT 2009,jff316,@ work missing church praying for the service...
1827423272,Sun May 17 10:40:46 PDT 2009,springcricket,yeah yeah @tpleeza that's what they ALL say......


## Checking for missing values NaN

In [30]:
print(f'There is {Train.isnull().any().sum()} column in train dataset with missing values.')#missing values in Train

There is 0 column in train dataset with missing values.


In [31]:
print(f'There is {Test.isnull().any().sum()} column in test dataset with missing values.')#missing values in Test

There is 0 column in test dataset with missing values.


# Data Wrangling

In [32]:
Train.head()

,target,date,user,text
id,,,,
2224496154,0,Thu Jun 18 09:34:25 PDT 2009,Deborah_Fay,Okay for some reason I still can't get the ba...
1687934301,4,Sun May 03 09:56:56 PDT 2009,haills,http://twitpic.com/4hjnj - Don't worry be happ...
1972073814,0,Sat May 30 08:30:21 PDT 2009,lechellealice,this movie is so sad
1557632618,4,Sun Apr 19 05:13:47 PDT 2009,janeslee,Good afternoon world. Been too busy to tweet. ...
1969750122,4,Sat May 30 01:05:20 PDT 2009,bvdaily,@QueensleyFelix LOL!We are just too classy I ...


In [33]:
Test.head()

,date,user,text
id,,,
1974447176,Sat May 30 13:09:18 PDT 2009,ugro,That's pretty much how I think it works with i...
1752584531,Sat May 09 22:20:24 PDT 2009,infinities,Holy awesome Leonard Nimoy! Also thought I sp...
2265878673,Sun Jun 21 07:29:15 PDT 2009,revybbcholic,@theodorainish yes they are I've tried to not...
1995188099,Mon Jun 01 13:05:22 PDT 2009,teddyrised,Millvina Dean the last known survivor of the ...
2071942859,Sun Jun 07 20:08:43 PDT 2009,kaylabeth_,I REALLY DO NOT LIKE THIS ROGER. Adam Pascal p...


In [0]:
def remove_punctuations(text):#For removing punctuation
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [0]:
def remove_URL_punctuation(tweet):#For removing Url links
    url = re.compile(r'https?://\S+|www\.\S+')
    text = url.sub(r'',tweet) 
    return remove_punctuations(text)

In [0]:
#Remove HashTag,links and punctuations
def remove_name(tweet):
    tweet_split = tweet.split()
    for word in tweet_split:
        if '@' in word or digits in word:
            tweet_split.remove(word)
            tweet = ' '.join(tweet_split)
    return remove_URL_punctuation(tweet).lower()

In [0]:
Train['text']=Train['text'].apply(remove_name)
Test['text']=Test['text'].apply(remove_name)

In [38]:
Train.head()

,target,date,user,text
id,,,,
2224496154,0,Thu Jun 18 09:34:25 PDT 2009,Deborah_Fay,okay for some reason i still cant get the bac...
1687934301,4,Sun May 03 09:56:56 PDT 2009,haills,dont worry be happy well that all depends on...
1972073814,0,Sat May 30 08:30:21 PDT 2009,lechellealice,this movie is so sad
1557632618,4,Sun Apr 19 05:13:47 PDT 2009,janeslee,good afternoon world been too busy to tweet wa...
1969750122,4,Sat May 30 01:05:20 PDT 2009,bvdaily,lolwe are just too classy i know i mustve gott...


In [39]:
Test.head()

,date,user,text
id,,,
1974447176,Sat May 30 13:09:18 PDT 2009,ugro,thats pretty much how i think it works with ip...
1752584531,Sat May 09 22:20:24 PDT 2009,infinities,holy awesome leonard nimoy also thought i spi...
2265878673,Sun Jun 21 07:29:15 PDT 2009,revybbcholic,yes they are ive tried to not to car or even t...
1995188099,Mon Jun 01 13:05:22 PDT 2009,teddyrised,millvina dean the last known survivor of the ...
2071942859,Sun Jun 07 20:08:43 PDT 2009,kaylabeth_,i really do not like this roger adam pascal pl...


In [0]:
#def stop_word_tweet(tweet):
#    tweet = word_tokenize(tweet)
#    return [word for word in tweet if word not in stopwords.words('english')]

In [0]:
#Train['text']=Train.apply(lambda x: stop_word_tweet(x.text),axis=1)

In [0]:
#Test['text']=Test['text'].apply(stop_word_tweet)

In [47]:
def stop_word_tweet(tweet):
    tweet = word_tokenize(tweet)
    lista = [word for word in tweet if word not in stopwords.words('english')]
    tweet = ' '.join(lista)
    return tweet


This example


In [0]:
Train['text']=Train.apply(lambda x: stop_word_tweet(x.text),axis=1)

In [56]:
Train.head()

,target,date,user,text
id,,,,
2224496154,0,Thu Jun 18 09:34:25 PDT 2009,Deborah_Fay,okay reason still cant get background picture ...
1687934301,4,Sun May 03 09:56:56 PDT 2009,haills,dont worry happy well depends situation
1972073814,0,Sat May 30 08:30:21 PDT 2009,lechellealice,movie sad
1557632618,4,Sun Apr 19 05:13:47 PDT 2009,janeslee,good afternoon world busy tweet washing sewing...
1969750122,4,Sat May 30 01:05:20 PDT 2009,bvdaily,lolwe classy know mustve gotten brooke amp don...


In [0]:
Test['text']=Test.apply(lambda x: stop_word_tweet(x.text),axis=1)

In [54]:
Test.head()

,date,user,text
id,,,
1974447176,Sat May 30 13:09:18 PDT 2009,ugro,thats pretty much think works iphone appsaccep...
1752584531,Sat May 09 22:20:24 PDT 2009,infinities,holy awesome leonard nimoy also thought spied ...
2265878673,Sun Jun 21 07:29:15 PDT 2009,revybbcholic,yes ive tried car even tink idk gue keinget te...
1995188099,Mon Jun 01 13:05:22 PDT 2009,teddyrised,millvina dean last known survivor titanic pass...
2071942859,Sun Jun 07 20:08:43 PDT 2009,kaylabeth_,really like roger adam pascal plz amp ty


# Visualization

In [0]:
data_labels = Train["target"].copy()#Target

In [0]:
#classes=[strongly negative (0), negative (1), neutral (2), positive (3), highly positive (4)]

In [0]:
classes, dist = np.unique(data_labels,return_counts=True)
num_class=len(classes)
print("Number of classes: {}".format(num_class))

In [0]:
plt.bar(classes,dist,align='center',alpha=0.5)
plt.show()

In [0]:
#class_weight ={c:dist[c]/np.sum(dist) for c in classes}

In [0]:
#tokens = [token for token in s.split(" ") if token != ""]

In [0]:
model = keras.models.Sequential([
keras.layers.Flatten(input_shape=[28, 28]),
keras.layers.Dense(300, activation="relu"),
keras.layers.Dense(100, activation="relu"),
keras.layers.Dense(10, activation="softmax")
])

In [0]:
model.summary()

In [0]:
keras.utils.plot_model(model)